This notebook demonstrates a game's data and analyse its various data

In [1]:
%load_ext autoreload
%autoreload 2

from statsnba import Game, Api
import requests_cache
import pandas as pd
pd.options.display.max_columns = 999

requests_cache.install_cache('test_cache')

In [2]:
api = Api()
game_ids = api.GetSeasonGameIDs('2009-10', 'Regular Season')
game_ids
sample_game_id = game_ids[0]

In [3]:
sample_game = Game(sample_game_id)
sample_game

Game(0020900292)

In [4]:
def matchup_to_df(matchups):
    lst = []
    for matchup in sample_game.Matchups:
        matchup_dict = {}
        _home_players = sorted([p.PlayerName for p in matchup.HomePlayers])
        _away_players = sorted([p.PlayerName for p in matchup.AwayPlayers])
        home_players = dict(zip(['h{}'.format(i) for i in range(5)], _home_players))
        away_players = dict(zip(['a{}'.format(i) for i in range(5)], _away_players))
        
        matchup_dict.update(home_players)
        matchup_dict.update(away_players)        
        home_boxscore = matchup.Boxscore.HomeTeamStats
        away_boxscore = matchup.Boxscore.AwayTeamStats
        
        matchup_dict.update({'home_{}'.format(k):v for k,v in home_boxscore.items()})
        matchup_dict.update({'away_{}'.format(k):v for k,v in away_boxscore.items()})
        lst.append(matchup_dict)
    return lst

df = pd.DataFrame(matchup_to_df(sample_game.Matchups))
print 'The first two rows of the matchups and their stats'
df.head(2)

The first two rows of the matchups and their stats


,a0,a1,a2,a3,a4,away_AST,away_BLK,away_DREB,away_ElapsedTime,away_FG3A,away_FG3M,away_FG3_PCT,away_FGA,away_FGM,away_FG_PCT,away_FTA,away_FTM,away_FT_PCT,away_OREB,away_PF,away_PLUS_MINUS,away_PTS,away_Possessions,away_REB,away_STL,away_TO,h0,h1,h2,h3,h4,home_AST,home_BLK,home_DREB,home_ElapsedTime,home_FG3A,home_FG3M,home_FG3_PCT,home_FGA,home_FGM,home_FG_PCT,home_FTA,home_FTM,home_FT_PCT,home_OREB,home_PF,home_PLUS_MINUS,home_PTS,home_Possessions,home_REB,home_STL,home_TO
0,Anthony Parker,JJ Hickson,LeBron James,Mo Williams,Shaquille O'Neal,0,0,1,00:03:13,1,0,0,5,1,0,0,0,0,0,2,-9,2,8,1,0,3,Andrew Bogut,Brandon Jennings,Carlos Delfino,Ersan Ilyasova,Luc Mbah a Moute,2,0,4,00:02:48,0,0,0,7,5,0,2,1,0,1,0,9,11,7,5,1,0
1,Anderson Varejao,Anthony Parker,JJ Hickson,LeBron James,Mo Williams,3,0,4,00:04:12,2,2,1,6,3,0,0,0,1,0,1,2,8,9,4,0,3,Andrew Bogut,Brandon Jennings,Carlos Delfino,Ersan Ilyasova,Luc Mbah a Moute,0,1,2,00:03:34,1,0,0,8,3,0,0,0,0,1,0,-2,6,8,3,0,1


In [5]:
print "Description of the various statistics"
df.describe()

Description of the various statistics


,away_AST,away_BLK,away_DREB,away_ElapsedTime,away_FG3A,away_FG3M,away_FG3_PCT,away_FGA,away_FGM,away_FG_PCT,away_FTA,away_FTM,away_FT_PCT,away_OREB,away_PF,away_PLUS_MINUS,away_PTS,away_Possessions,away_REB,away_STL,away_TO,home_AST,home_BLK,home_DREB,home_ElapsedTime,home_FG3A,home_FG3M,home_FG3_PCT,home_FGA,home_FGM,home_FG_PCT,home_FTA,home_FTM,home_FT_PCT,home_OREB,home_PF,home_PLUS_MINUS,home_PTS,home_Possessions,home_REB,home_STL,home_TO
count,34.000000,34.000000,34.000000,34,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
mean,0.617647,0.176471,1.176471,0 days 00:01:15.264705,0.294118,0.176471,0.088235,2.176471,1.029412,0.205882,0.794118,0.735294,0.088235,0.235294,0.558824,0.441176,2.970588,2.941176,1.411765,0.235294,0.529412,0.323529,0.117647,0.794118,0 days 00:01:08.882352,0.588235,0.117647,0.088235,2.735294,1.029412,0.088235,0.470588,0.352941,0.088235,0.323529,0.617647,-0.441176,2.529412,3.029412,1.117647,0.176471,0.352941
std,1.128547,0.458627,1.641703,0 days 00:01:38.705042,0.675521,0.520524,0.287902,3.520250,1.585642,0.410426,1.066839,1.053388,0.287902,0.605971,0.894128,3.341123,3.880685,3.757346,2.090638,0.495973,0.991946,0.588814,0.327035,1.387813,0 days 00:01:39.302343,0.891633,0.409338,0.287902,3.518857,1.604639,0.287902,1.022039,0.733706,0.287902,0.638207,0.888127,3.341123,3.586467,3.415520,1.871067,0.458627,0.597081
min,0.000000,0.000000,0.000000,0 days 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0 days 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0 days 00:00:08.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.750000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0 days 00:00:08,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.750000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0 days 00:00:53.500000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0 days 00:00:44.500000,0.000000,0.000000,0.000000,1.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,2.000000,0.500000,0.000000,0.000000
75%,1.000000,0.000000,1.750000,0 days 00:01:34.250000,0.000000,0.000000,0.000000,2.750000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.750000,4.000000,3.000000,2.000000,0.000000,0.750000,0.750000,0.000000,1.000000,0 days 00:01:38,1.000000,0.000000,0.000000,4.000000,1.000000,0.000000,0.750000,0.000000,0.000000,0.750000,1.000000,0.750000,4.000000,4.000000,1.000000,0.000000,1.000000
max,5.000000,2.000000,8.000000,0 days 00:08:30,3.000000,2.000000,1.000000,19.000000,8.000000,1.000000,4.000000,4.000000,1.000000,3.000000,4.000000,9.000000,19.000000,19.000000,11.000000,2.000000,3.000000,2.000000,1.000000,6.000000,0 days 00:08:44,4.000000,2.000000,1.000000,19.000000,7.000000,1.000000,5.000000,3.000000,1.000000,3.000000,3.000000,9.000000,15.000000,18.000000,9.000000,2.000000,2.000000
